In [1]:
####Importation des packagins :

import pandas as pd
import numpy as np

#Packaging de visualisation
import seaborn as sns
import matplotlib.pyplot as plt

#packaging preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#selection de model
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

#packaging model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

#packagng metrics
from sklearn.metrics import make_scorer, mean_squared_error



import warnings
warnings.filterwarnings('ignore')

In [2]:
laptop = pd.read_csv('laptop_data.csv', index_col = 0)

In [3]:
df = pd.read_csv('laptop_cleaned.csv')

In [4]:
X,y = df.drop('Price', axis = 1), df['Price']

# 1. RandomForest

In [5]:
forest = RandomForestRegressor()
cross = KFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
model_init = cross_val_score(estimator=forest, 
                X=X,y=y, 
                cv=cross,  
                scoring=make_scorer(mean_squared_error))

# 2. Mesure du score 

In [7]:
#### definition d'une fonction calculant le rmse : 
def rmse(score):
    moy = score.mean()
    rmse = np.sqrt(moy)
    return round(rmse,2)

In [8]:
rmse(model_init)
#deja un meilleur score de base que knn amélioré

148.68

# 3. Amelioration du modèle :

- On va tenter d'améliorer :
    - KFold n_split
    - parametres du randomforest (n_estimators, max_depth , min_samples_split)

In [9]:
parametre_grid = {"n_estimators":[100, 200, 300,400,500]}

In [10]:
grid = GridSearchCV(estimator=RandomForestRegressor(), 
                    param_grid=parametre_grid, 
                    cv=cross)

In [11]:
grid.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [100, 200, 300, 400, 500]})

In [12]:
best_grid = grid.best_estimator_
best_grid

RandomForestRegressor(n_estimators=500)

In [13]:
best_model_grid = cross_val_score(estimator= best_grid, 
                X=X,y=y, 
                cv=cross,  
                scoring=make_scorer(mean_squared_error))

In [14]:
dict_score_split = {}
for i in range(2,20,2):
    cross = KFold(n_splits=i, shuffle=True, random_state=42)
    best_model_grid = cross_val_score(estimator= best_grid, 
                X=X,y=y, 
                cv=cross,  
                scoring=make_scorer(mean_squared_error))
    dict_score_split[i] = float(rmse(best_model_grid))
    print("split ",i, ": rmse =", dict_score_split[i], "$")

split  2 : rmse = 149.31 $
split  4 : rmse = 151.9 $
split  6 : rmse = 148.97 $
split  8 : rmse = 147.43 $
split  10 : rmse = 144.19 $
split  12 : rmse = 145.65 $
split  14 : rmse = 144.28 $
split  16 : rmse = 142.95 $
split  18 : rmse = 142.44 $


In [15]:
best_n_split = min(dict_score_split, key = dict_score_split.get)
best_n_split

18

In [16]:
best_cross = KFold(n_splits=best_n_split, shuffle = True, random_state=42)

In [17]:
best_model = cross_val_score(estimator=best_grid, X = X, y = y, cv = best_cross, scoring=make_scorer(mean_squared_error))

In [18]:
print("Prédiction du prix avec l'amélioration du modèle à ", rmse(best_model), "$ près") 

Prédiction du prix avec l'amélioration du modèle à  142.59 $ près


In [19]:
print("Amélioration de la prédiction du prix de :", round(float(rmse(model_init))-float(rmse(best_model)),2), "$")

Amélioration de la prédiction du prix de : 6.09 $


#### Conclusion : 
- RandomForest est le modèle le plus précis de notre pool, nous allons l'utiliser pour notre déploiment.

## Sauvegarde du modèle 

In [30]:
import pickle

In [31]:
file = open('laptop_prediction.pkl', 'wb')
pickle.dump(best_grid, file)
file.close()

In [32]:
laptop.head(1)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832


## Exemple de prediction : 

In [28]:
best_grid.predict([X.iloc[1,:]])

array([419.6780428])